In [37]:
from functools import reduce
import numpy as np
import math

clicks = []
with open('c1.txt','r') as f:
    for line in f:
        clicks.append(reduce(lambda a,b:int(a)+int(b),line.split()))
clicks1 = np.array(clicks)

clicks = []
with open('c2.txt','r') as f:
    for line in f:
        clicks.append(reduce(lambda a,b:int(a)+int(b),line.split()))
clicks2 = np.array(clicks)

In [44]:
ctr1 = []
for i in range(len(clicks1)//1000):
    ctr1.append((clicks1[i*1000:(i+1)*1000]>0).sum()/1000)
ctr1 = np.array(ctr1)

ctr2 = []
for i in range(len(clicks2)//1000):
    ctr2.append((clicks2[i*1000:(i+1)*1000]>0).sum()/1000)
ctr2 = np.array(ctr2)

print(ctr1.mean(),ctr2.mean())

0.699946 0.670108


In [40]:
def t_stat(x1,x2):
    return (abs(x1.mean()-x2.mean())/
            (math.sqrt(x1.std()/len(x1)+
                       x2.std()/len(x2))))
 
N = 1000
t_base = 1.96

count = 0
for _ in range(N):
    x1 = np.random.choice(ctr1,len(ctr1),replace=True)
    x2 = np.random.choice(ctr2,len(ctr2),replace=True)
    
    if t_stat(x1,x2)>t_base:
        count+=1

print(f'ASL = {count/N}')

ASL = 1.0


***В выборках почти наверняка есть статистичкое различие в CTR метрике***

***Теперь оставим только те запросы, куда были клики и посчитаем среди них количесто запросов в которые было более 1 (2) кликов***

In [50]:
ctr1 = []
clicks1 = clicks1[clicks1>0]
for i in range(len(clicks1)//1000):
    ctr1.append((clicks1[i*1000:(i+1)*1000]>1).sum()/1000)
ctr1 = np.array(ctr1)

ctr2 = []
clicks2 = clicks2[clicks2>0]
for i in range(len(clicks2)//1000):
    ctr2.append((clicks2[i*1000:(i+1)*1000]>1).sum()/1000)
ctr2 = np.array(ctr2)
print(ctr1.mean(),ctr2.mean())

N = 1000
t_base = 1.96

count = 0
for _ in range(N):
    x1 = np.random.choice(ctr1,len(ctr1),replace=True)
    x2 = np.random.choice(ctr2,len(ctr2),replace=True)
    
    if t_stat(x1,x2)>t_base:
        count+=1

print(f'ASL = {count/N}')

0.7430114449213162 0.24616865671641794
ASL = 1.0


In [51]:
ctr1 = []
for i in range(len(clicks1)//1000):
    ctr1.append((clicks1[i*1000:(i+1)*1000]>2).sum()/1000)
ctr1 = np.array(ctr1)

ctr2 = []
for i in range(len(clicks2)//1000):
    ctr2.append((clicks2[i*1000:(i+1)*1000]>2).sum()/1000)
ctr2 = np.array(ctr2)
print(ctr1.mean(),ctr2.mean())

N = 1000
t_base = 1.96

count = 0
for _ in range(N):
    x1 = np.random.choice(ctr1,len(ctr1),replace=True)
    x2 = np.random.choice(ctr2,len(ctr2),replace=True)
    
    if t_stat(x1,x2)>t_base:
        count+=1

print(f'ASL = {count/N}')

0.3999685264663805 0.038617910447761195
ASL = 1.0


***Выборки отличаются в обеих этих метриках и это отличие статистически значимо***

***Выбрать лучший сплит имея только клики довольно сложно, но я попробую<br><br>
&emsp;Среднее CTR больше у первого сплита. Это значит, что в первом сплите появилось больше запросов на которые мы отвечаем достаточно хорошо, чтобы пользователь на него кликнул. Но этого недостаточно. По сути мы повысили полноту, но ничего не знаем о точности. Возможно, что на то, на что мы отвечали раньше хорошо, теперь мы отвечаем плохо. Да и впринципе то, что пользователь кликает на документ в общем случае не означает, что этот документ релевантен<br><br>
&emsp;Для того, чтобы оценить точность я буду исходить из следующего предположения - в идеальном поисковике пользователь кликает лишь на один документ. Почему это так? Разберём возможные ситуации: пользователь кликнул на один документ и ушёл. Есть 3 возможности:<br>
   &emsp;&emsp;1) Он нашёл, что искал и ушёл<br>
   &emsp;&emsp;2) Он не нашёл, что искал, но все остальные наши результаты совсем плохие, поэтому он просто ушёл<br>
   &emsp;&emsp;3) Рандом. Отошёл от компьютера, забыл что искал, умер за клавиатурой<br>
&emsp;Считаем, что шанс 3) довольно мал. Чем лучше наш поисковик, тем меньше вероятность 2). Таким образом, если у нас довольно неплохой поисковик, то 1 клик обычно означает, что поиск отработал хорошо.<br><br>
   &emsp;Теперь допустим пользователь кликнул на много документов. Опять 3 возможности:<br>
   &emsp;&emsp;1) Пользователь собирает много разной информации<br>
   &emsp;&emsp;2) Пользователь всё никак не может найти ответ на свой вопрос<br>
   &emsp;&emsp;3) Рандом. Кошка по клавиатуре ходит и кликает на сайты<br>
&emsp;Считаем, что шанс 3) довольно мал. Обычно пользователи нетерпеливые и не любят долго искать информацию, поэтому будем считать, что 1) не очень частотный. Таким образом в среднем много кликов означает, что пользователь не может найти ответ на свой вопрос. В интернете ответ почти наверняка есть, а значит проблема в поисковике.<br>
   &emsp;Таким образом мы хотим, чтобы было как можно больше запросов в которых мало кликов и как можно меньше запросов в которых кликов много. Эту статистику я уже посчитал выше и по ней видно, что во втором сплите количество запросов с более, чем одним кликом ЗНАЧИТЕЛЬНО меньше, чем в первом сплите. По CTR различие не такое фундаментальное, поэтому я считаю, что лучшим является второй сплит***

***Ответ: лучшим является второй сплит***